In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 5 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Méthodes de résolution de systèmes linéaires itératives 
#    Préconditionnement
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import time
import numpy as np
from scipy.sparse import diags

# Préconditionnement

## Equation de Poisson

On souhaite résoudre le problème elliptique constitué par l'équation de Poisson soumise à des conditions aux limites de type de Dirichlet :

$$
\left\{
\begin{aligned}
-\Delta u(x) & =  b(x) \quad \text{ dans } \; \Omega = [0,1] \quad \text{avec} \; b(x)=1\\
        u(0) & =  0 \quad \text{ sur }  \;  \partial \Omega
\end{aligned}
\right.
$$

## Méthode de Tchebyschef

La méthode itérative de Tchebyschef permet aussi de converger vers une solution du problème.

In [ ]:
def chebychev(a, b, lambda1, lambda2, max_iter, eps=1.e-6, output=False):

    theta = (lambda2 + lambda1) / 2.
    delta = (lambda2 - lambda1) / 2.

    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    rk = b - a.dot(xk)
    sigma = theta/delta
    rhok = 1./sigma
    dk = (1./theta)*rk

    for k in range(max_iter):
        xk = xk + dk
        rk = rk - a.dot(dk)
        norm_rk = np.linalg.norm(rk)
        if norm_rk/norm_b < eps: break
        rhokm1 = rhok
        rhok = 1./(2.*sigma-rhok)
        dk = rhok*rhokm1*dk + ((2*rhok)/delta)*rk

    if output:
        print(f"  Nombre d'itérations = {k+1}")
        print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

### Cas 1d

In [ ]:
nx = 100
dx = 1/(nx+1)
diagonals = [np.repeat(2/(dx*dx), nx), np.repeat(-1/(dx*dx), nx-1), np.repeat(-1/(dx*dx), nx-1)]
a = diags(diagonals, [0, -1, 1])

b = np.ones(nx)

print("\nRésolution par la méthode du Tchebyschev")
cst = (4/(dx**2))
# L'intervalle de valeurs propres est pris dans son intégralité
# cf. polycopié

lambda1 = cst*np.sin((np.pi)/(2*(nx+1)))**2
lambda2 = cst*np.sin((np.pi*nx)/(2*(nx+1)))**2
u = chebychev(a, b, lambda1, lambda2, max_iter=10000, output=True)

## Méthode du gradient conjugué préconditionné

In [ ]:
def conjugate_gradient(a, b, eps=1.e-6):
    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    rk = b - a.dot(xk)
    pk = rk
    rkm1 = rk

    for k in range(b.size):
        apk = a.dot(pk)
        alpha = np.dot(rk,rk) / np.dot(pk, apk)
        xk = xk + alpha*pk
        rk = rk - alpha*apk
        norm_rk = np.linalg.norm(rk)
        #print(k, np.linalg.norm(rk))
        if norm_rk/norm_b < eps: break
        beta = np.dot(rk,rk) / np.dot(rkm1, rkm1)
        pk = rk + beta*pk
        rkm1 = rk

    print(f"  Nombre d'itérations = {k+1}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

def prec_cheb_conjugate_gradient(a, b, lambda1, lambda2, eps=1.e-6, iter_prec=10):

    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)
    rk = b - a.dot(xk)
    rkm1 = rk
    zk = chebychev(a, rk, lambda1, lambda2, max_iter=iter_prec)
    zkm1 = zk
    pk = zk

    for k in range(b.size):
        apk = a.dot(pk)
        alpha = np.dot(rk,zk) / np.dot(pk, apk)
        xk = xk + alpha*pk
        rk = rk - alpha*apk
        norm_rk = np.linalg.norm(rk)
        #print('prec', k, np.linalg.norm(rk))
        if norm_rk/norm_b < eps: break
        zk = chebychev(a, rk, lambda1, lambda2, max_iter=iter_prec)
        beta = np.dot(rk,zk) / np.dot(rkm1, zkm1)
        pk = zk + beta*pk
        rkm1 = rk
        zkm1 = zk

    print(f"  Nombre d'itérations = {k+1}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

### Cas 1d

In [ ]:
nx = 10000
dx = 1/(nx+1)
diagonals = [np.repeat(2/(dx*dx), nx), np.repeat(-1/(dx*dx), nx-1), np.repeat(-1/(dx*dx), nx-1)]
a = diags(diagonals, [0, -1, 1])

b = np.ones(nx)

print("\nRésolution par la méthode du gradient conjugué")
u = conjugate_gradient(a, b)

print("\nRésolution par la méthode du gradient conjugué preconditionné avec Tchebychev")
cst = (4/(dx**2))
# estimation heuristique d'un "bon" intervalle de valeurs propres à considérer  - Cf. Polycopié
# Estimation uniquement valable pour le préconditionnement ! 
i = 70
alpha = cst*np.sin((np.pi*i)/(2*(nx+1)))**2
beta = cst*np.sin((np.pi*(nx-(i-1)))/(2*(nx+1)))**2
u = prec_cheb_conjugate_gradient(a, b, alpha, beta, iter_prec=10)

### Cas 2d

In [ ]:
nx = 600
ny = nx
dx = 1/(nx+1)
dy = 1/(ny+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2 + 2/dy**2, nx*ny)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny)
diag_y = np.repeat(-1/dy**2, nx*(ny-1))
a = diags([diag, diag_x, diag_x, diag_y, diag_y], [0, -1, 1, -nx, nx])

# second membre
b = np.ones(nx*ny)

print(f"Cas 2d : nx = {nx} et ny = {ny} => nx . ny = {nx*ny}")

print("\nRésolution par la méthode du gradient conjugué")
ucg = conjugate_gradient(a, b)

print("\nRésolution par la méthode du gradient conjugué preconditionné avec Tchebychev")
cst = (8/(dx**2))
# estimation heuristique d'un "bon" intervalle de valeurs propres à considérer  - Cf. Polycopié
# Estimation uniquement valable pour le préconditionnement ! 
i = 5
alpha = cst*np.sin((np.pi*i)/(2*(nx+1)))**2
beta = cst*np.sin((np.pi*(nx-(i-1)))/(2*(nx+1)))**2
u = prec_cheb_conjugate_gradient(a, b, alpha, beta, iter_prec=10)

### Cas 3d

In [ ]:
nx = 100
ny = nx
nz = nx
dx = 1/(nx+1)
dy = 1/(ny+1)
dz = 1/(nz+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2 + 2/dx**2 + 2/dz**2, nx*ny*nz)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny*nz)
diag_y = np.tile(np.repeat([-1/dy**2, 0.], (nx*(ny-1), nz)), nz)
diag_z = np.repeat(-1/dz**2, nx*ny*(nz-1))
a = diags([diag, diag_x, diag_x, diag_y, diag_y, diag_z, diag_z], [0, -1, 1, -nx, nx, -nx*ny, nx*ny])

# second membre
b = np.ones(nx*ny*nz)

print(f"Cas 3d : nx = {nx}, ny = {ny} et nz = {nz} => nx . ny . nz = {nx*ny*nz}")
print("\nRésolution par la méthode du gradient conjugué")
ucg = conjugate_gradient(a, b)

print("\nRésolution par la méthode du gradient conjugué preconditionné avec Tchebychev")
cst = (12/(dx**2))
# estimation heuristique d'un "bon" intervalle de valeurs propres à considérer  - Cf. Polycopié
# Estimation uniquement valable pour le préconditionnement ! 
i = 4
alpha = cst*np.sin((np.pi*i)/(2*(nx+1)))**2
beta = cst*np.sin((np.pi*(nx-(i-1)))/(2*(nx+1)))**2
u = prec_cheb_conjugate_gradient(a, b, alpha, beta, iter_prec=10)